# Bases de dades

La base de dades [ENA](https://www.ebi.ac.uk/ena/browser/home) ofereix la possibilitat de realitzar consultes de forma *programàtica*: des d'un *script* o des de la línia de comandaments en un terminal tipus *BASH*. En un terminal, els programes `curl` i `wget` permeten descarregar continguts de la xarxa. A l'entorn d'R també disposem d'un paquet anomenat `curl` que ens permet descarregar informació d'internet.

L'objectiu d'aquesta pràctica és composar un *script* d'R que represente gràficament l'evolució de la quantitat de dades de seqüències curtes en ENA en els últims anys. És a dir, volem reproduir i actualitzar el gràfic **Reads growth** mostrat a les [estadístiques d'ENA](https://www.ebi.ac.uk/ena/browser/about/statistics).

Aquest document t'ofereix un exemple dels passos a seguir, que hauràs d'editar per tal que l'script execute la funció desitjada. És a dir, utilitza els blocs de codi d'aquest document com a plantilla per confeccionar el teu propi script. Pots obrir un quadern jupyter nou (`File` $\rightarrow$ `New Notebook` $\rightarrow$ `R`). Si coneixes RStudio i prefereixes fer-ho allà, també pots obrir una sessió d'RStudio des d'aquesta sessió en mybinder.org.

## Llibreria *curl*

Primer, carreguem el paquet *curl* i l'utilitzem per descarregar la documentació més recent de la manera de consultar ENA mitjançant la línia de comandaments:

In [ ]:
library('curl')
curl_download(url = 'https://www.ebi.ac.uk/ena/portal/api/doc?format=pdf',
              destfile = 'apiEna.pdf')

El paquet *curl* ofereix altres funcions per descarregar continguts. Utilitzarem `curl_download`, que sempre guarda els continguts descarregats en un arxiu.

## L'API d'ENA

Una *interície de programació d'aplicacions* (API) és la forma de comunicar-se dues aplicacions. En aquest cas: la base de dades d'ENA i el nostre script. L'API d'ENA consisteix en les regles que tradueixen una cerca concreta en una adreça d'internet o *URL*. Aquesta URL es composa de:

1. L'adreça del protal `htpps://www.ebi.ac.uk/ena/portal/api`.
2. L'*endpoint* `/search`, que és el que ens permet buscar alguna cosa.
3. Els paràmetres de cerca, separats per "&" i que inclouen:
    - *result*: aquest paràmetre és obligatori i especifica quin tipus de dades busquem: *read_run*, *read_experiment*, *sample*, *study*, *sequence_release*, *wgs_set*, etc.
    - *query*: (opcional) condicions o filtres de cerca units mitjançant "AND", "OR" o "NOT" i entre cometes.
    - *fields*: (opcional) llista de camps (separats per comes) que es desitja obtenir (específics del tipus de dades que busquem; pot incloure: *scientific_name*, *collection_date*, *strain*, etc.
    - *limit*: (opcional) número màxim de registres que seran descarregats. El 0 significa "tots".
    - *format*: (opcional) el format per defecte és *tsv*, però es pot sol·licitar *json*.

Podem obtenir una llista dels valors possibles del paràmetre *result* mitjançant: `https://www.ebi.ac.uk/ena/portal/api/results?dataPortal=ena`.

Per a un tipus de dades concret de la llista (e.g., *sample*), podem conéixer els camps d'informació que podem utilitzar com a filtres o termes de cerca (*query*) mitjançant: `https://www.ebi.ac.uk/ena/portal/api/searchFields?result=sample`.

Així mateix, els camps disponibles per l'informe de resultats es consulten així: `https://www.ebi.ac.uk/ena/portal/api/returnFields?result=sample`.

Per últim, per especificar valors exigits a alguns termes de cerca (*query*), necessitem conéixer els valors permesos (o el *vocabulari controlat*): `https://www.ebi.ac.uk/ena/portal/api/controlledVocab?field=mol_type`.

Per fer el codi més clar i fàcil d'editar, en el bloc de codi de sota definisc individualment les diferents parts de la URL que vull construir i després les ajunte amb la funció `paste()`:

In [ ]:
portal   <- 'https://www.ebi.ac.uk/ena/portal/api/'
endpoint <- 'search?'
# Busque genomes assemblats...
result   <- 'result=assembly'
# ...i complets de peixos actinopterigis (NCBI taxon id 7898):
query    <- '&query=tax_tree(7898) AND genome_representation="full"'
fields   <- '&fields=version,tax_id,scientific_name,last_updated,base_count'
limit    <- '&limit=0'

# Ara ho junte tot i comprove que queda bé:
URL <- paste0(portal, endpoint, result, query, fields, limit, sep='', collapse='')
URL

Sembla que hem construït la URL de forma correcta. Tanmateix, si intentes descarregar el seu contingut en un arxiu (`curl_download(URL, destfile='z1.tsv')`) obtindràs un error. El motiu és que en la URL apareixen espais i caràcters reservats ("=", "(", ")") on no s'esperaven. El problema està en el fragment `tax_tree(7898) AND genome_representation="full"`. Només en aqeusta part cal substituir els caràcters reservats per la seua representació en [codi percent](https://es.wikipedia.org/wiki/C%C3%B3digo_porciento). Per tant, cal definir `query` de nou per arreglar-ho:

In [ ]:
query  <- paste('&query=',
                URLencode('tax_tree(7898) AND genome_representation="full"',
                          reserved = TRUE),
                sep = '', collapse = '')
URL <- paste(portal, endpoint, result, query, fields, limit, sep='', collapse='')
URL

Fixa't en l'ús de les funcions `paste()` i `URLencode()`. Pots aprendre sobre els seu funcionament mitjançant `help(URLencode)`.

### Exercici 1
Tenint en compte el que has aprés, construeix ara una consulta en format URL per descarregar: la data de publicació, el nombre de lectures i el nombre de bases dels 100000 primers registres de tipus *read_run*. (Limitem el nombre de registres per no saturar la RAM).

## Guardar i carregar els resultats

In [ ]:
curl_download(URL, destfile = 'fish.tsv')

Amb açò ha d'haver aparegut l'arxiu `fish.tsv` a la carpeta de treball. Pot semblar poc eficient utilitzar R per descarregar un arxiu i després tornar a llegir-lo en R. Especialment quan la funció `curl_fetch_memory()` carregaria directament el contingut de la URL en la memòria de treball. Però és més fàcil així.

Ara, doncs, toca llegir l'arxiu descarregat, utilitzant la funció `read.table()`:

In [ ]:
fish <- read.table('fish.tsv',
             header = TRUE,
             colClasses = c('character','numeric','numeric','character','Date','numeric'),
             sep = '\t',
             na.strings = '')
head(fish, n = 12)

Observa les primeres línies de les dades que hem carregat en memòria. L'objecte `fish` és un *data frame*: un tipus de taula en R en què cada columna és una variable.

### Exercici 2
Descarrega els resultats de la cerca confeccionada en l'**exercici 1** i carrega'ls en la memòria o espai de treball d'R. 

## Comptar les bases per data
Per representar gràficament l'acumulació de registres i del nombre de bases total al llarg del temps, necessitem fer algunes operacions. Concretament, per cada data (columna *last updated*) hem de sumar el nombre de bases totals (*base_count*) i comptar el nombre de registres actualitzat en eixa data. Comencem per ací. Aquest tipus d'operacions que s'apliquen per separat a cada tros d'una taula i després es reuneixen en una nova taula es poden fer prou bé amb el paquet `plyr`:

In [ ]:
library(plyr)
# La funció ddply() de plyr pren com a dades inicials un data frame
# i produeix un altre data frame.
PerData <- ddply(.data = fish,
                 .variables = 'last_updated',
                 .fun = function(x) data.frame(bases = sum(x$base_count),
                                               registres = nrow(x)))
head(PerData)

Si tens interés, pots aprendre a utilitzar el paquet `plyr` en aquest [tutorial](http://swcarpentry.github.io/r-novice-gapminder/12-plyr/index.html).

Hi ha altres maneres de fer-ho. Per exemple, amb la funció `aggregate()`, disponible per defecte, que aplica una funció a cada tros d'un *data frame* delimitat per les files que tenen el mateix valor d'un factor (o la mateixa combinació de valors de més d'un factor). Per exemple, per sumar el nombre de bases seqüenciades en la mateixa data, podem fer:

In [ ]:
BasesPerData <- aggregate(fish$base_count, by = list(date=fish$last_updated), sum)
head(BasesPerData)

Utilitzarem la taula `PerData`, creada amb `ddply()`, que conté el nombre de registres i el nombre de bases que van ser actualitzades en cada data. Podem representar-ho gràficament:

In [ ]:
plot(PerData$last_updated, PerData$bases, type='l')

In [ ]:
head(PerData[order(PerData$bases, decreasing=TRUE),])

En ordenar les línies de la taula `PerData`, veiem fàcilment en quines dates s'actualitzaren més registres. **Sabries dir a quines espècies corresponen els 505 registres del 29 d'octubre de 2020?**

### Exercici 3
Utilitza la funció `ddply()` (o bé la funció `aggregate()`) per agrupar el nombre de lectures curtes i el nombre de bases publicades a ENA en cada data.

## Suma acumulada

El que necessitem ara és afegir a la taula `PerData` una columna amb la suma *acumulada* de bases i la de registres. Podem comprovar que la taula està de fet ordenada per dates i aleshores aplicar la funció `cumsum()`:

In [ ]:
is.unsorted(PerData$last_updated)

In [ ]:
PerData$basesAcumulades    <- cumsum(PerData$bases)
PerData$registresAcumulats <- cumsum(PerData$registres)
tail(PerData)

A dalt muestre el final de la taula amb la funció `tail()` en lloc de mostrar el principi, perquè vull veure si la suma acumulada ha funcionat. **Què creus que haguer passat si en la taula original `fish` hi haguera cap registre amb un nombre desconegut `NA` de bases?**

A continuació, ho representem gràficament, utilitzant una escala logarítmica per al nombre de bases acumulades.

In [ ]:
plot(PerData$last_updated, PerData$basesAcumulades,
     type = 'l', log = 'y',
     xlab = 'Data', ylab = 'Nombre de bases acumulades',
     main = 'Genomes de peixos ossis')

### Exercici 4
Calcula i representa gràficament el nombre acumulat de lectures curtes i de bases al llarg del temps amb els 100000 primers registres de *read_run* que has descarregat d'ENA.

## Per concloure

- Quins avantatges té realitzar les consultes de forma programàtica, en lloc de fer-ho mitjançant la interfície gràfica?
- Quins problemes creus que podria haver en executar de nou l'script d'ací a 10 anys? I d'ací a 100 anys?
- Has *anotat* el teu script amb comentaris? Per què?
- Coneixes alguna eina per poder mostrar, compartir i editar un script de forma col·laborativa i remota amb altres persones?